In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader



In [ ]:
# Definir el bloque residual

In [11]:
class BasicBlock(nn.Module):
    """
    La variable expansion se usa en la arquitectura de ResNet para definir cuántos canales de salida tendrá el bloque residual en relación con su entrada.
    Esto significa que la cantidad de canales de salida es la misma que la cantidad de canales internos en el bloque.

    Ejemplo en ResNet-18:

    Si entran 64 canales, la salida del bloque también tiene 64 canales.
    Si entran 128 canales, la salida también tiene 128 canales.
    No hay aumento en los canales.
    """
    expansion = 1  # Para ResNet-18 y ResNet-34

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        """
        nn.Conv2d: Crea una capa convolucional 2D; in_channels, representa el número de canales de entrada, por ejemplo una imagen RGB tiene 3 canales; out_channels es el número de filtros que se aplicaran tal que cada uno genera una salida; kernel_size=3 , es el tamaño del filtro; stride indica el desplazamiento del filtro en cada paseo; por ejemplo si el stride es de 1 significa que el filtro se mueve 1 pixel a la vez. Un stride 2 reduce a la mitad el tamaño espacial, el padding=1 agrega 1 pixel de borde (cero) alrededor de la imagen de entrada para conservar su tamaño original al aplicar una convolución y finalmente el bias=False, indicamos que no existirá sesgo; esto es muy común cuando se usa BatchNorm luego de la convolución.
        """
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        """
        nn.BatchNorm2d: Normaliza la salida de una capa convolucional para cada mini-lote, canal por canal; es decir, toma la salida de la convolución y le aplica una transformación para estabilizar el entrenamiento
        """
        """
        ✅ Propósito principal
            Estabiliza el entrenamiento al mantener activaciones con media cercana a 0 y varianza cercana a 1.
            Acelera la convergencia (entrena más rápido).
            Reduce el problema de covariate shift interno, es decir, los cambios en la distribución de activaciones dentro del modelo.
            Permite usar tasas de aprendizaje más altas sin que el entrenamiento explote.
        """
        """
         ¿Por qué out_channels?
            Porque la normalización se aplica por canal, y la salida de la convolución anterior tiene out_channels mapas de activación (uno por filtro). Entonces:
        """
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        """
        Si el canal de entrada y el canal de salidad es el mismo o el stride sigue siendo 1 el shortcut es solo  una identidad;  caso contrario se aplica una convolucional 1x1 con el stride recibido como argumento para que la forma coincida con la salida
        """
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        """
        ¿Por qué revisamos stride != 1?
        El stride en la primera convolución del bloque puede ser diferente de 1 (normalmente 2), lo que significa que la imagen se reduce de tamaño (submuestreo).

        👉 Ejemplo:
        Si la entrada tiene tamaño [batch, 64, 32, 32] y usas:

        nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        La salida será de tamaño [batch, 128, 16, 16]
        ⛔ Pero tu identity = x sigue teniendo tamaño [batch, 64, 32, 32]
        ⛔ ¡No puedes sumarlos directamente!

        Entonces, en este caso, necesitas ajustar la shortcut para que también reduzca la resolución con stride=2.
        ¿Por qué revisamos in_channels != out_channels?
        Incluso si el tamaño espacial es el mismo (stride=1), puede cambiar el número de canales.
        Por ejemplo:

        nn.Conv2d(64, 128, kernel_size=3, stride=1)
        Entonces:

        Salida del bloque → [batch, 128, H, W]
        Entrada (x) → [batch, 64, H, W] ⛔ Nuevamente, no puedes sumarlos.
        📌 Necesitas una convolución 1×1 que cambie los canales de la entrada (x) a 128.
        :param x:
        :return:
        """
        identity = self.shortcut(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = nn.ReLU()(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity  # Suma residual
        out = nn.ReLU()(out)
        return out


# Construir la red ResNet-18

In [13]:
class ResNet18(nn.Module):
    def __init__(self, num_classes=10):  # CIFAR-10 tiene 10 clases
        super(ResNet18, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self._make_layer(64, 64, 2, stride=1)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.layer3 = self._make_layer(128, 256, 2, stride=2)
        self.layer4 = self._make_layer(256, 512, 2, stride=2)
        """
        ✅ Aplica pooling global adaptativo para convertir cualquier mapa de activación de tamaño arbitrario a un tamaño fijo de 1×1 por canal.
        En concreto:

        Si la entrada tiene forma [batch_size, canales, alto, ancho]
        La salida será: [batch_size, canales, 1, 1]
        Es decir:

        Promedia todos los valores espaciales (alto × ancho) dentro de cada canal
        Pero lo hace de manera automática, sin que tengas que especificar el tamaño exacto de la entrada
        """
        """
        🧩 ¿Por qué "adaptativo"?
            Porque funciona con cualquier tamaño de entrada.

            No necesitas saber si la entrada será 4×4, 8×8 o 7×7
            Siempre la va a reducir a 1×1
            Muy útil cuando la arquitectura puede variar o la entrada no es de tamaño fijo
        """
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, in_channels, out_channels, blocks, stride):
        layers = []
        layers.append(BasicBlock(in_channels, out_channels, stride))
        for _ in range(1, blocks):
            layers.append(BasicBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        """
        La función torch.flatten(input, start_dim) aplana (convierte en vector) las dimensiones a partir de start_dim hasta el final.
        """
        """
        Ejemplo sea x.shape = [128, 512, 1, 1]
        Necesitamos convertirlo en [128, 512] para poder pasarlo a una nn.Linear (que espera un vector plano).
        x = torch.flatten(x, 1)
        produce x.shape = [128, 512]
        """
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

# Crear un modelo ResNet-18 desde cero
#model = ResNet18()
#print(model)


# ------------------------------------------------ FIN de creación de la red ResNet-18 ------------------------------------------------

# Carga de datos CIFAR-10

In [14]:
# Configurar GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Transformaciones para normalizar los datos y aplicar data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # Aumentación de datos
    transforms.RandomHorizontalFlip(),  # Flip aleatorio
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))  # Normalización
])
"""
Los números que se usan en la normalización de CIFAR-10 tienen un propósito específico y no son elegidos al azar. Vamos a explicarlo bien.
transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
Esta línea normaliza cada canal (R, G, B) de las imágenes, de la siguiente forma: pixel_normalized = (pixel - mean) / std

"""


transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

# Cargar dataset CIFAR-10
batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
# Definir clases de CIFAR-10
classes = ('avión', 'automóvil', 'pájaro', 'gato', 'ciervo', 'perro', 'rana', 'caballo', 'barco', 'camión')

Usando dispositivo: cpu


# Cargar el modelo ResNet-18

In [15]:
# Cargar el modelo en el dispositivo
model = ResNet18(num_classes=10).to(device)

# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()  # Pérdida para clasificación
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)  # Adam con regularización L2

# Usar un Scheduler para reducir la tasa de aprendizaje con el tiempo
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Entrenamiento del modelo

In [17]:
num_epochs = 20  # Número de épocas

for epoch in range(num_epochs):
    model.train()  # Modo entrenamiento
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Resetear gradientes
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Calcular pérdida
        loss.backward()  # Backpropagation
        optimizer.step()  # Actualizar pesos

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    scheduler.step()  # Actualizar learning rate

    epoch_loss = running_loss / len(trainloader)
    epoch_acc = 100 * correct / total
    print(f"Época [{epoch+1}/{num_epochs}], Pérdida: {epoch_loss:.4f}, Precisión: {epoch_acc:.2f}%")


Época [1/20], Pérdida: 0.5216, Precisión: 82.16%
Época [2/20], Pérdida: 0.4819, Precisión: 83.57%
Época [3/20], Pérdida: 0.4570, Precisión: 84.37%
Época [4/20], Pérdida: 0.4373, Precisión: 85.04%
Época [5/20], Pérdida: 0.2948, Precisión: 90.17%
Época [6/20], Pérdida: 0.2518, Precisión: 91.47%
Época [7/20], Pérdida: 0.2320, Precisión: 92.16%
Época [8/20], Pérdida: 0.2131, Precisión: 92.73%
Época [9/20], Pérdida: 0.1995, Precisión: 93.19%
Época [10/20], Pérdida: 0.1884, Precisión: 93.60%
Época [11/20], Pérdida: 0.1787, Precisión: 93.93%
Época [12/20], Pérdida: 0.1638, Precisión: 94.40%
Época [13/20], Pérdida: 0.1539, Precisión: 94.72%
Época [14/20], Pérdida: 0.1491, Precisión: 95.01%
Época [15/20], Pérdida: 0.1205, Precisión: 96.02%
Época [16/20], Pérdida: 0.1139, Precisión: 96.33%
Época [17/20], Pérdida: 0.1114, Precisión: 96.29%
Época [18/20], Pérdida: 0.1049, Precisión: 96.53%
Época [19/20], Pérdida: 0.1033, Precisión: 96.61%
Época [20/20], Pérdida: 0.1012, Precisión: 96.66%


# Evaluación del modelo en datos de prueba

In [18]:
model.eval()  # Modo evaluación
correct = 0
total = 0

with torch.no_grad():  # Desactivar gradientes para evaluación
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Precisión en el conjunto de prueba: {accuracy:.2f}%")


Precisión en el conjunto de prueba: 91.59%
